Environment set-up

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Set the matplotlib style for consistent plotting aesthetics
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Specify the path to the dataset directory
database_path = r'database'

# Get a list of CSV files from the specified directory
files = Path(database_path).glob('*.csv')

# Initialize an empty list to hold individual ETF dataframes
dfs = []

for f in files:
    data = pd.read_csv(f)
    data['ETF_name'] = f.stem
    dfs.append(data)
    
# Concatenate the list of ETF dataframes into a single dataframe
etf = pd.concat(dfs, ignore_index=True)


# Display basic information about the merged dataframe
etf.info()


In [ ]:
# Convert the 'Date' column to datetime type and ensure 'ETF_name' is treated as a string
etf['Date'] = pd.to_datetime(etf['Date'], utc=True)
etf = etf.astype({'ETF_name': 'string'})

# Display updated information about the dataframe
etf.info()

In [ ]:
#Selecting and printing columns
etf.columns

Initial Diagonistics

In [ ]:
#Selecting unique ETF and setting the index. 
unique_etf = etf.index.unique()


# Add new columns for year, month, and day extracted from the 'Date' column
etf['Year'] = etf['Date'].dt.year
etf['Month'] = etf['Date'].dt.month
etf['Day'] = etf['Date'].dt.day

#Check the result
etf.head()

Calculate Mean Price and Summarize by ETF
The average price is calculated by trading parameters such as 'Open', 'High', 'Low', and 'Close'

In [ ]:
#Calculate the mean of 'Open', 'High', 'Low', and 'Close' columns and add it as a new column#mean_by_etf = pd.DataFrame()
etf['mean'] = etf.loc[:, 'Open':'Close'].mean(axis=1)
etf.groupby('ETF_name')['mean'].describe()


Mapping Colors to ETFs and Visualizing Trading Volume and Visualizing ETF Trading Volume

In [ ]:
# Create a list of colors corresponding to unique ETFs
COLORS = ('orange', 'lightgreen', 'lightblue',
          'red', 'yellow', 'brown', 'darkgreen', 'darkblue', 'black', 'purple')
etf_colors = dict(zip(unique_etf, COLORS))

# Group ETFs by name and calculate the total trading volume
volume_sum_by_etf = etf.groupby('ETF_name')['Volume'].sum()


# Plot a bar chart to visualize total trading volume for each ETF
ax = volume_sum_by_etf.plot(kind='bar', color=COLORS, legend=False, grid=True)
ax.set_ylabel('Total Volume')
ax.set_title('ETF Volume by ETF Name')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
volume_sum_by_etf


Individual ETF Volume Analysis

In [ ]:
# Plot volume analysis for each ETF
for etf_name in etf['ETF_name'].unique():
    etf_subset = etf[etf['ETF_name'] == etf_name]
    
    # Create a subplot for volume analysis
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot trading volume
    ax.plot(etf_subset['Date'], etf_subset['Volume'],
            linewidth = 1, label='Trading Volume', color='blue')
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Volume')
    ax.set_title(f'Trading Volume Analysis for {etf_name}')
    ax.legend()
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Visualizing ETF Mean Prices Over Time for all ETF's

In [ ]:
# Pivot the DataFrame to have each ETF's mean price as columns
pivot_etf = etf.pivot_table(
    index='Date', columns='ETF_name', values='mean', aggfunc='mean')

# Plot a separate line for each ETF
ax2 = pivot_etf.plot.line(
    linewidth = 1, legend=True, figsize=(10, 6), color=COLORS)
ax2.set_xlabel('Year')
ax2.set_ylabel('Mean Price')
ax2.set_title('Mean Price of ETFs Over Time')
plt.xticks(rotation=45)
ax2.legend(bbox_to_anchor=(1, 0.8))
plt.tight_layout()
plt.show()


Visualizing ETF Mean Prices Over Time for each ETF

In [ ]:
# Create separate line plots for each ETF
for etf_name in etf['ETF_name'].unique():
    etf_subset = etf[etf['ETF_name'] == etf_name]
       # # Plotting the line plot for the current ETF

    ax = etf_subset.plot.line(x='Date', y='mean', linewidth=1, figsize=(10, 6))
    ax.set_xlabel('Year')
    ax.set_ylabel('Mean Price')
    ax.set_title(f'Mean Price of {etf_name} Over Time')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

A bar plot is created to visualize the volatility of each ETF.

In [ ]:
# Calculate daily returns
etf['Daily_Return'] = etf.groupby('ETF_name')['Close'].pct_change()

# Calculate annualized volatility (assuming 252 trading days in a year)
etf_volatility = etf.groupby('ETF_name')['Daily_Return'].std() * np.sqrt(252)

# Visualize volatility
ax = etf_volatility.plot(kind='bar', color=COLORS, legend=False, grid=True)
ax.set_ylabel('Volatility')
ax.set_title('Volatility of ETFs')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Calculating and Visualizing Correlation Matrix

* The code calculates the correlation matrix between daily returns of different ETFs.
* The correlation matrix is visualized as a heatmap, showing the correlation values between ETF pairs.

In [ ]:
# Calculate daily returns
etf['Daily_Return'] = etf.groupby('ETF_name')['Close'].pct_change()

# Calculate correlation matrix
correlation_matrix = etf.pivot_table(
    index='Date', columns='ETF_name', values='Daily_Return').corr()

# Visualize correlation matrix as a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of ETFs')
plt.show()

Return Analysis


Calculating and Analyzing Price Differences

* The code calculates the price difference and percentage difference between the first and last closed prices for each ETF.
* The percentage difference is calculated as `(last_closed_price - first_closed_price) / first_closed_price * 100`.
* The ETF name and price difference are printed.

In [ ]:
#Calculating the difference between the first closed price, and the last
for etf_name in etf['ETF_name'].unique():
    etf_subset = etf[etf['ETF_name'] == etf_name]
    
    # Get the first and last closed prices
    first_closed_price = etf_subset['Close'].iloc[0]
    last_closed_price = etf_subset['Close'].iloc[-1]

    # Calculate the difference
    price_difference = last_closed_price - first_closed_price
    percentage_difference = price_difference/first_closed_price*100
    percentage_difference = percentage_difference.round(2)
    
    # Print the ETF name and the price difference
    print(f'ETF: {etf_name}, Price Difference: {percentage_difference}%')
   

Calculating Returns

* The code calculates daily returns for each ETF using the percentage change in the 'Close' prices.
* Annualized volatility is calculated for each ETF using the standard deviation of daily returns multiplied by the square root of 252 (assuming 252 trading days in a year).

In [ ]:

# Plot individual return analysis for each ETF
for etf_name in etf['ETF_name'].unique():
    etf_subset = etf[etf['ETF_name'] == etf_name].copy()
    
    # Calculate simple returns
    etf_subset['simple_return'] = (etf_subset.loc[:,'Close'] -etf_subset.loc[:,'Open']) / etf_subset.loc[:,'Open']
    
    # Calculate logarithmic returns
    etf_subset['log_return'] = np.log(etf_subset.loc[:,'Close'] / etf_subset.loc[:,'Open'])
    
    # Calculate cumulative returns
    etf_subset['cumulative_return'] = (1 + etf_subset['simple_return']).cumprod()
    
    # Create subplots for each ETF's return analysis
    fig, axes = plt.subplots(2, 1, figsize=(10, 8))
    
    # Plot simple returns
    ax = etf_subset.plot(x='Date', y='simple_return',linewidth=1, ax=axes[0])
    ax.set_xlabel('Date')
    ax.set_ylabel('Simple Return')
    ax.set_title(f'Simple Return Analysis for {etf_name}')
    ax.legend().set_visible(False)
    
    # Plot cumulative returns
    ax = etf_subset.plot(x='Date', y='cumulative_return',linewidth=1, ax=axes[1], color='orange')
    ax.set_xlabel('Date')
    ax.set_ylabel('Cumulative Return')
    ax.set_title(f'Cumulative Return Analysis for {etf_name}')
    ax.legend().set_visible(False)
    
    plt.tight_layout()
    plt.show()